In [1]:
# Original Max-Min Parents and Children ALgorithm
# input: data X and significance level alpha
# output: Parents and Children set for a node

In [22]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2

In [84]:
X = pd.read_csv("/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Batches/censored_Erk_Target/batch_0.csv")

# each cell can be -1, 0 or 1
#Pre-processing data
X['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1), X['Pvalue']))
del X['Unnamed: 0']
del X['TrueIndex']
del X['CauseGene']
del X['EffectGene']
del X['Replicate']
del X['Treatment']
PC=[]
for column in X:
    if (column != "Pvalue"):
        X[column] = list(map(lambda x: x+1 , X[column]))

# preliminary checkup
#counter = 0
#for column in X:
#    if column!="Pvalue" and counter<530:
#        del X[column]
#        counter+=1

In [70]:
def ColumnLookup(X,column):
    listX=arrayX(X)
    listC=list(column)
    for i in range(1,541):
        true = 0
        for j in range(0,5997):
            if (listX[i][j]!=column[j]):
                true+=1
                break
        if (true==0):
            return i
    return -1

In [81]:
def ExistDseparator(TargetNode,Xi, Z, X, alpha):
    flagExist = False
    dsepSet=[]
    
    for i in range(0,(2**len(Z))-1):
        IDsubsetZ_dec = i
        IDsubsetZ_bin = bin(IDsubsetZ_dec)
        subsetZ = getZsubset(IDsubsetZ_bin,Z)
        # no cache
        dep = Dep(TargetNode,Xi,subsetZ, X, alpha)
        if (dep==0):
            flagExist = True
            dsepSet = subsetZ
            break
    return [flagExist,dsepSet]

In [25]:
def GetCondValues(z, sizes):
    if len(sizes)>1:
        maxz = np.sum(sizes)
        d = np.multiply(np.ones((f1,len(sizes))),0)
        
        div_dim = maxz/sizes[0]
        num2div = z-1
        
        for i in range(0,len(sizes)-1):
            d[i] = floor(num2div/div_dim)
            num2div=mod(num2div,div_dim)
            div_dim = div_dim/sizes[i+1]
        d[i+1] = num2div
    else:
        d = z-1
    return d

In [26]:
def Dep(TargetNode, Xi, CondVars, X, alpha):
    #print('First line of Dep')
    vvTarget = [0,1]
    vvXi = [0,1,2]
    if len(X[0])*3 <= (5 * 3*3*(3**len(CondVars))):
        return 1
    if (len(CondVars)==0):
        S = np.multiply(np.ones((len(vvTarget),len(vvXi))),0)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                var1=np.array(X[0]) # Target Node
                var2=np.array(Xi)
                # looking up for 0, 1 and 2
                op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                S[i][j]=np.sum(op1*op2)
        G2=0
        N = np.sum(S)
        #print(S)
        #print(N)
        Si =np.sum(S,axis=1)
        Sj =np.sum(S,axis=0)
        Df = ((len(vvTarget)-1)*(len(vvXi)-1))
        Dedf = np.sum(np.sum(S[np.where(S>0)])\
                      - np.sum(Si[np.where(Si>0)])\
                      - np.sum(Sj[np.where(Sj>0)]) -1)
        if (Dedf<1):
            Dedf=1
            #print('Here is the trouble')
            #return 0
        #print(S)
        for k in range(0,3):
            for l in range(0,3):
                #print(i)
                #print(j)
                if (S[i][j]>0):
                    #print(Si[i])
                    #print(Sj[j])
                    G2 = G2 + S[i][j]*np.log((S[i][j])*N/(Si[i]*Sj[j]))
                    #print(G2)
        G2 = 2*G2
        
    else: # test conditional dependency
        szCondVars = 3**len(CondVars)
        S = np.multiply(np.ones((len(vvTarget),len(vvXi),szCondVars)),0)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    condValue = GetCondValue(k,np.multiply(np.ones((len(CondVars),1)),3))
                    flagDataCondVars = np.ones((len(vvXi),1))
                    for l in range(0,len(CondVars)):
                        X_l = X[CondVars(l)]
                        #flagDataCondVars = flagDataCondVars*(X_l[where(X_l==condValue[l])])
                        flagDataCondVars = np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l.tolist())))
                    var1=np.array(X[0])
                    var2=np.array(Xi)
                    # looking up for -1, 0 and 1
                    op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                    #print(op1)
                    op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                    #print(op2)
                    S[i][j][k]=np.sum(op1*op2)*flagDataCondVars
        G2 = 0
        Sjk = np.sum(S,axis=0)
        Sik = np.sum(S,axis=1)
        Sk = np.sum(Sjk,axis=1)
        #print(Sjk)
        #print(Sik)
        #print(Sk)
        Dedf = np.sum(np.sum(np.sum(S[np.where(S>0)]))) -\
                np.sum(np.sum(Sik[np.where(Sik>0)])) -\
                np.sum(np.sum(Sjk[np.where(Sjk>0)])) +\
                np.sum(Sk[np.where[Sk>0]])
        if Dedf>1:
            Dedf=1
            #print('No, here is the trouble')
            #return 0
        
        # compute G2 statistic
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXI)):
                for k in range(0,szCondVars):
                    if (S[i][j][k]>0):
                        G2=G2+(S[i][j][k] * log( (S[i][j][k]) * Sk[0][0][k])/\
                                    (Sik[i][0][k] * Sjk[0][j][k]))
        G2 = 2*G2
        
    assoc = (alpha - (1 - chi2.cdf(G2,Dedf)))/alpha
    #print(G2)
    #print(Dedf)
    #print(assoc)
    if assoc<0:
        assoc=0
    return assoc
    

In [7]:
def getZsubset(bin_id,Z):
    bin_str=str(bin_id)
    Zsubset=[]
    for i in range(0,len(bin_str)-2):
        if bin_str[i]==1:
            Zsubset.append(Z[i])
    return Zsubset

In [8]:
def arrayUniverse(TargetNode,X):
    Universe = []
    for key in X:
        if (key!=TargetNode):
            Universe.append(X[key].copy(deep=True).tolist())
    return Universe

In [64]:
def arrayX(X):
    returnable=[]
    for key in X:
        #tagged_c = MB_Column(X[key].copy(deep=True).tolist(),key)
        returnable.append(X[key].copy(deep=True).tolist())
    return returnable

In [37]:
def MinAssoc(TargetNode, Xi,Z, fixedCondVars, X, alpha):
    subsetZ_assoc=[]
    min_assoc=999
    if len(Z)==0:
        min_assoc = Dep(TargetNode, Xi, fixedCondVars, X, alpha)
        subsetZ_min_assoc = fixedCondVars
        #print(min_assoc)
    else:
        for IDsubsetZ_dec in range(0,2**len(Z)-1):
            IDsubsetZ_bin = bin(IDsubsetZ_dec)
            subsetZ = getZsubset(IDsubsetZ_bin,Z)
            #print(len(Xi))
            subsetZ_assoc.append(Dep(TargetNode, Xi, fixedCondVars+subsetZ,X,alpha))
            #print(subsetZ_assoc[IDsubsetZ_dec])
            if subsetZ_assoc[IDsubsetZ_dec] < min_assoc:
                min_assoc = subsetZ_assoc[IDsubsetZ_dec]
                subsetZ_min_assoc = fixedCondVars + subsetZ
                if (min_assoc==0):
                    break
    return min_assoc

In [35]:
def MaxMinHeuristic(TargetNode, CPC, Universe, X, alpha):
    F=[]
    assocF=-1
    Z = CPC
    fixedCondVars = []
    if (len(CPC)>0):
        Z = CPC[0:-1]           # all but the last one   
        fixedCondVars = CPC[-1] # we use last one
    
    for i in range(0,len(Universe)):
        if (len(Universe[i])==0):
            continue
        minAssoc = MinAssoc(TargetNode,Universe[i], Z,fixedCondVars, X,alpha)
        #print(minAssoc)
        if minAssoc>assocF:
            assocF = minAssoc
            F=Universe[i]
        if minAssoc == 0:
            Universe[i]=[]
        #print(F)
    return [F,assocF,Universe]

In [39]:
def MMPC_(TargetNode,Universe,X,alpha):
    
    CPC=[]
    # Phase I: Foward
    print("Entering Phase I")
    while len(Universe)>0:
        CPC_old = list(CPC) # copy
        maxminheur=MaxMinHeuristic(TargetNode,CPC,list(Universe),X,alpha)
        # maxminheur = [F, assocF, Universe]
        F = maxminheur[0]
        assocF = maxminheur[1]
        Universe = maxminheur[2]
        #print("Universe printing line")
        #print(Universe)
        print(CPC)
        if assocF > 0:
            CPC.append(F)
            indF=Universe.index(F)
            Universe.pop(indF)
        if (len(CPC)==len(CPC_old)):
            break
    
    # Phase 2: Backward
    print("Entering Phase II")
    if len(CPC)>1:
        print("CPC size was larger than 1")
        for i in range(0,len(CPC)):
            Z=list(CPC)
            # index is i
            Z[i]=[]
            if ExistDseparator(TargetNode,CPC[i],Z,X,alpha)[0] == True:
                CPC[i]=[]
    return CPC

In [30]:
def MMPC(TargetNode, X, alpha):
    # The universe will be an array of DataFrame Columns
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [82]:
alpha=0.05
PC=MMPC('Pvalue',X,alpha)
print(PC)

10
11
Entering Phase I
[]
[[1, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 0, 2, 2, 2, 1, 1, 2, 0, 1, 2, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 1, 1, 1, 0, 2, 2, 0, 2, 1, 1, 1, 0, 2, 0, 1, 0, 2, 2, 2, 1, 0, 2, 2, 2, 2, 0, 2, 2, 0, 1, 1, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 1, 2, 2, 1, 2, 2, 0, 2, 1, 2, 2, 2, 0, 2, 1, 2, 1, 1, 0, 2, 1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 0, 0, 1, 2, 1, 0, 2, 2, 0, 1, 2, 0, 2, 2, 1, 0, 1, 1, 1, 2, 2, 2, 1, 0, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 1, 2, 0, 2, 0, 1, 2, 1, 2, 2, 2, 0, 2, 0, 1, 1, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 1, 0, 2, 0, 1, 1, 0, 0, 2, 2, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 1, 0, 1, 2, 2, 0, 0, 2, 0, 1, 1, 2, 0, 1, 2, 0, 0, 1, 2, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 2, 1, 1, 1, 1, 0, 0, 1, 2, 2, 

[[1, 0, 2, 0, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 0, 2, 2, 2, 1, 1, 2, 0, 1, 2, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 1, 1, 1, 0, 2, 2, 0, 2, 1, 1, 1, 0, 2, 0, 1, 0, 2, 2, 2, 1, 0, 2, 2, 2, 2, 0, 2, 2, 0, 1, 1, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 1, 2, 2, 1, 2, 2, 0, 2, 1, 2, 2, 2, 0, 2, 1, 2, 1, 1, 0, 2, 1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 0, 0, 1, 2, 1, 0, 2, 2, 0, 1, 2, 0, 2, 2, 1, 0, 1, 1, 1, 2, 2, 2, 1, 0, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 1, 2, 0, 2, 0, 1, 2, 1, 2, 2, 2, 0, 2, 0, 1, 1, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 1, 0, 2, 0, 1, 1, 0, 0, 2, 2, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 1, 0, 1, 2, 2, 0, 0, 2, 0, 1, 1, 2, 0, 1, 2, 0, 0, 1, 2, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 2, 1, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2,

In [83]:
results = []
for e in PC:    
    print(ColumnLookup(X,e))

1
2
3
4
5
6
7
8
9
10


In [ ]:
X.head()

In [ ]:
fff*fff

In [ ]:
def modify_list(arr):
    print(arr.pop())
    print(arr.pop())
    return

arr=[1,2,3]
print(arr)
modify_list(arr)
print(arr)
    

In [ ]:
rrr= np.multiply(np.ones((2,2)),0)
rrr[np.where(rrr==1)]

In [ ]:
b=np.array(data)

In [ ]:
b[1]

In [ ]:
b[2]

In [ ]:
b[3]

In [ ]:
c=b[0]

In [ ]:
c

In [ ]:
c[np.where(c<1)].size

In [ ]:
rrr=np.array([1,2,2,1])
ccc=np.array([3,2,4,1])

In [ ]:
rrr+ccc

In [ ]:
a=[1,2,2]
b=[2,3,4]
a+b

In [ ]:
a=1+2+4\
  +5

In [ ]:
rrr*3